# Tutorial: Word2Vec in TensorFlow Basic

This is a basic version of implementation. It does NOT scale on big dataset.

- Original tutorial link: https://www.tensorflow.org/get_started/os_setup#anaconda_installation
- Installation of TensorFlow: https://www.tensorflow.org/get_started/os_setup#anaconda_installation

- Word2Vec has two flavors, Skip-gram and CBOW. We use Skip-gram in this tutorial.

## Skip-gram vs CBOW

1. Skip-gram models the conditional probability as $P(\text{context_word}\quad | \quad \text{target_word})$, while CBOW models the conditional probability in an inverse way, i.e. $P(\text{target_word} \quad | \quad \text{context})$.

## Data set

http://mattmahoney.net/dc/text8.zip

After unzip, it is a single text file of size ~95M, it is a big one-line text document. The following is the first 1000 bytes in the file (print whole takes a long time):

     anarchism originated as a term of abuse first used against early working class radicals including the diggers of the english revolution and the sans culottes of the french revolution whilst the term is still used in a pejorative way to describe any act that used violent means to destroy the organization of society it has also been taken up as a positive label by self defined anarchists the word anarchism is derived from the greek without archons ruler chief king anarchism as a political philosophy is the belief that rulers are unnecessary and should be abolished although there are differing interpretations of what this means anarchism also refers to related social movements that advocate the elimination of authoritarian institutions particularly the state the word anarchy as most anarchists use it does not imply chaos nihilism or anomie but rather a harmonious anti authoritarian society in place of what are regarded as authoritarian political structures and coercive economic instituti

The data seems to be pre-processed already, e.g. punctuation removed, convert to lower cases.

## Data Preparation

The raw data is usually a corpus (i.e. a collection of articles/reviews/...). There are a few data preprosessing steps before you can learn the embeddings.

- Cleaning data to remove any noisy/non-informative elements, e.g. stop words maybe. 
- Integerizing the cleaned text corpus with a indexed vocabulary. That is, each word in the vocabulary is assgined a unique integer as its index (e.g. 1-> father, 2-> mother, 3 - loves, ... ), then replace each word in the corpus with the corresponding index. (e.g. if the corpus has the sentence "father loves mother", then after integerization, it becomes "1 3 2".) Details can be found in https://www.tensorflow.org/code/tensorflow/examples/tutorials/word2vec/word2vec_basic.py.
- Some words are very rare in the corpus, we might not want to model them. So one way to handle rare words is to replace them with a common token, e.g. UNKOWN. Then the embedding is learned for UNKOWN.

## Input Format for Skip-gram Algorithm

- A batch full of integers representing the source context words.
- Target words.


In [101]:
import tensorflow as tf
import collections
import numpy as np
import random
import math

## Step 1: Read Data

In [16]:
# We already unzipped the data as a single text file.
# tf.compat.as_str converts input into a string
with open('text8', 'r') as f:
    words = tf.compat.as_str(f.read()).split()

In [17]:
# Print a few words in the dataset
print words[:10]

# So data is a list of words.

['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first', 'used', 'against']


## Step 2: Pre-process Data

In [20]:
vocab_size = 50000  # only keep top 50,000 words (in terms of frequency)

In [18]:
count = [['UNKOWN', -1]]  

In [22]:
# keep top 49,999 words from the original dataset, 
# leave 1 for UNKNOWN.
count.extend(collections.Counter(words).most_common(vocab_size - 1))

In [23]:
count[:10]

[['UNKOWN', -1],
 ('the', 1061396),
 ('of', 593677),
 ('and', 416629),
 ('one', 411764),
 ('in', 372201),
 ('a', 325873),
 ('to', 316376),
 ('zero', 264975),
 ('nine', 250430)]

In [24]:
w2idx = dict()
for word, _ in count:
    w2idx[word] = len(w2idx)  # assign a unqiue index (1,2,...)

In [28]:
data = []
unkown_count = 0

# Integerize text document (replace word with index)
for word in words:
    if word in w2idx:
        index = w2idx[word]
    else:
        index = 0
        unkown_count += 1
    data.append(index)

count[0][1] = unkown_count

In [29]:
idx2w = dict( zip(w2idx.values(), w2idx.keys()) )  # reverse key-value (make key as value, and value as key)

In [32]:
del words  # no need anymore (save memory)

In [33]:
print('Most common words (+UNK)', count[:5])

('Most common words (+UNK)', [['UNKOWN', 418391], ('the', 1061396), ('of', 593677), ('and', 416629), ('one', 411764)])


In [35]:
print('Sample data', data[:10], [idx2w[i] for i in data[:10]])

('Sample data', [5239, 3084, 12, 6, 195, 2, 3137, 46, 59, 156], ['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first', 'used', 'against'])


### Preapre a training batch



In [91]:
# Preapre a training batch

data_index = 0 

def generate_batch(batch_size, num_skips, skip_window):
    global data_index
    assert batch_size % num_skips == 0
    assert num_skips <= 2 * skip_window
    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    span = 2*skip_window + 1  #  total number of words in a context, i.e. [skip_window, target, skip_window]
    buffer = collections.deque(maxlen=span)  # list-like container with fast appends and pops on either end
    for _ in range(span):
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)  # go through from first word to the last in the data
    for i in range(batch_size//num_skips):
        target = skip_window  # middle word in the span
        targets_to_avoid = [skip_window]
        for j in range(num_skips):
            while target in targets_to_avoid:
                target = random.randint(0, span-1)  # why random sample?
            targets_to_avoid.append(target)
            batch[i*num_skips + j] = buffer[skip_window]  # middle word is always the predictor
            labels[i*num_skips + j, 0] = buffer[target]
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)
    # Backtrack a little bit to avoid skipping words in the end of a batch
    data_index = (data_index + len(data) - span) % len(data)
    return batch, labels

In [93]:
# - num_skip: how many samples for a target word.

batch, labels = generate_batch(batch_size=8, num_skips=2, skip_window=2)
for i in range(8):
    print(batch[i], idx2w[batch[i]],
        '->', labels[i, 0], idx2w[labels[i, 0]])

(3137, 'abuse', '->', 195, 'term')
(3137, 'abuse', '->', 2, 'of')
(46, 'first', '->', 3137, 'abuse')
(46, 'first', '->', 2, 'of')
(59, 'used', '->', 3137, 'abuse')
(59, 'used', '->', 46, 'first')
(156, 'against', '->', 59, 'used')
(156, 'against', '->', 128, 'early')


## Step 3: Construct the Model

In [99]:
batch_size = 128
embd_size = 128  # Dimension of the embedding vector.
skip_window = 1       # How many words to consider left and right.
num_skips = 2         # How many times to reuse an input to generate a label.

In [96]:
# We pick a random validation set to sample nearest neighbors. Here we limit the
# validation samples to the words that have a low numeric ID, which by
# construction are also the most frequent.
valid_size = 16     # Random set of words to evaluate similarity on.
valid_window = 100  # Only pick dev samples in the head of the distribution.
valid_examples = np.random.choice(valid_window, valid_size, replace=False)

In [95]:
num_neg_sampled = 64    # Number of negative examples to sample.

In [104]:
graph = tf.Graph()

with graph.as_default():
    # Input data
    # Initilize the placeholder for the batch input (e.g. each batch
    # could be a sentence, so the size is the number of works in the
    # sentence)
    train_inputs = tf.placeholder(tf.int32, shape=[batch_size])

    # Initialize the placeholder for the labels, it has to be a column
    # vector. Each word in a batch (e.g. sentence) can be a label
    train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
    
    valid_dataset = tf.constant(valid_examples, dtype=tf.int32)
    
    with tf.device('/cpu:0'):
        # Initialize the matrix (with random numbers) for holding embeddings. 
        # Each row is an embedding vector for a word.
        embeddings = tf.Variable( tf.random_uniform([vocab_size, embd_size], -1.0, 1.0) )
        
        # Look up the vector for each of the source words in the batch.
        # This is to associate each word with its embedding vector 
        # after estimation.
        embd = tf.nn.embedding_lookup(embeddings, train_inputs)
        
        # Initialize a weight matrix to hold the parameters during estimation.
        # Each row is a coeffiient vector in a logistic regression.
        weights = tf.Variable( tf.truncated_normal([vocab_size, embd_size], stddev=1.0/math.sqrt(embd_size)) )
        # Initialize the bias / intercept vector. Each element is an 
        # intercept in a logistic regression.
        biases = tf.Variable( tf.zeros([vocab_size]) )
        
        # Construct optimization target
        # - num_sampled: k, i.e. number of noise words sampled from a vocabulary distribution
        loss = tf.reduce_mean( tf.nn.nce_loss(weights, biases, embd, train_labels, num_neg_sampled, vocab_size) )
        
        # Construct the optimizer
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=1.0).minimize(loss)
        
        # Compute the cosine similarity between minibatch examples and all embeddings.
        norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
        normalized_embeddings = embeddings / norm
        valid_embeddings = tf.nn.embedding_lookup(
          normalized_embeddings, valid_dataset)
        similarity = tf.matmul(
          valid_embeddings, normalized_embeddings, transpose_b=True)

        # Add variable initializer.
        init = tf.global_variables_initializer()

The optimization target is formulated as the log-likelihood function for a logistic regresion model. Remember, our goal is to maximize the following two probabilities for a context word: 

$$P(\text{a context word shows up } | \text{ target_word})$$ 
$$P(\text{a context word does NOT show up } | \text{a randomly sampled noise word} )$$

The above model update the parameters based on one batch of the data. We will need loop it over different batches of the data set.

## Step 4: Train the Model

In [105]:
num_steps = 100001

In [107]:
with tf.Session(graph=graph) as session:
    # We must initialize all variables before we use them.
    init.run()
    print("Initialized")
    
    avg_loss = 0
    for step in xrange(num_steps):
        # Generate a new batch of data
        batch_inputs, batch_labels = generate_batch(batch_size, num_skips, skip_window)
        feed_dict = {train_inputs: batch_inputs, train_labels: batch_labels}
        
        _, loss_val = session.run([optimizer, loss], feed_dict=feed_dict)
        avg_loss += loss_val
        
        if step % 2000 == 0:
            if step > 0:
                avg_loss /= 2000
            # The average loss is an estimate of the loss over the last 2000 batches.
            print("Average loss at step ", step, ": ", avg_loss)
            avg_loss = 0

        # Note that this is expensive (~20% slowdown if computed every 500 steps)
        if step % 10000 == 0:
          sim = similarity.eval()
          for i in xrange(valid_size):
            valid_word = idx2w[valid_examples[i]]
            top_k = 8  # number of nearest neighbors
            nearest = (-sim[i, :]).argsort()[1:top_k + 1]
            log_str = "Nearest to %s:" % valid_word
            for k in xrange(top_k):
              close_word = idx2w[nearest[k]]
              log_str = "%s %s," % (log_str, close_word)
            print(log_str)
            
    final_embeddings = normalized_embeddings.eval()

Initialized
('Average loss at step ', 0, ': ', 270.61480712890625)
Nearest to s: whispering, dogg, managers, manoeuvre, headstone, xvi, hh, fractions,
Nearest to were: selector, constructible, dizziness, excerpts, suharto, inadvertent, auctions, charenton,
Nearest to their: refrigerator, clerihew, commuted, unintended, citation, bust, sade, tetris,
Nearest to state: peta, alley, microcontroller, measure, not, dignity, plausibility, wiping,
Nearest to war: cedes, illumination, graviton, serial, starved, knighted, breadth, hemorrhage,
Nearest to b: taupin, compensating, ominous, explained, bioses, chairperson, privileged, dessau,
Nearest to nine: reggaeton, lfflin, lain, behavioral, pelletier, marines, whatever, grasso,
Nearest to they: had, darkness, carla, duplicating, livy, traders, aliyah, enid,
Nearest to years: reining, affordable, hydrazine, opaque, sundial, communion, gia, sousa,
Nearest to called: untimely, ulrike, protesting, binary, ada, prisms, lothar, emb,
Nearest to an: ove

## Step 6: Visualization